In [27]:
import pandas as pd
import pymongo
import requests
import json

In [2]:
bike_data_csv = "data/metro-bike-share-trips-2018-q3.csv"

In [3]:
bike_data_df = pd.read_csv(bike_data_csv)

In [6]:
bike_data_df.head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type
0,94851140,8,2018-07-01 00:04:00,2018-07-01 00:12:00,3058,34.035801,-118.233170,3082,34.046520,-118.237411,6279,30,One Way,Monthly Pass
1,94851141,8,2018-07-01 00:04:00,2018-07-01 00:12:00,3058,34.035801,-118.233170,3082,34.046520,-118.237411,6518,365,One Way,Flex Pass
2,94851138,15,2018-07-01 00:09:00,2018-07-01 00:24:00,4147,34.145248,-118.150070,4174,34.165291,-118.150970,4823,365,One Way,Monthly Pass
3,94851137,7,2018-07-01 00:22:00,2018-07-01 00:29:00,4157,34.140999,-118.132088,4162,34.147499,-118.148010,6115,30,One Way,Monthly Pass
4,94851136,35,2018-07-01 00:23:00,2018-07-01 00:58:00,3013,33.779819,-118.263023,3013,33.779819,-118.263023,12055,1,Round Trip,Walk-up


In [14]:
items = bike_data_df[:5].to_json(orient='records')

In [19]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [20]:
db = client.bike_data_db
collection = db.bike_trip

In [21]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bike_data_db'), 'bike_trip')

In [28]:
items_db = json.loads(items)

In [38]:
db.bike_trip.insert_many(items_db)

In [39]:
collection.count_documents({})

5

In [32]:
collection.find_one()

{'_id': ObjectId('5bf72c8d8a95a662949378f7'),
 'trip_id': 94851140,
 'duration': 8,
 'start_time': '2018-07-01 00:04:00',
 'end_time': '2018-07-01 00:12:00',
 'start_station': 3058,
 'start_lat': 34.035801,
 'start_lon': -118.23317,
 'end_station': 3082,
 'end_lat': 34.04652,
 'end_lon': -118.237411,
 'bike_id': 6279,
 'plan_duration': 30,
 'trip_route_category': 'One Way',
 'passholder_type': 'Monthly Pass'}

In [34]:
oneWayDoc = collection.find({"trip_route_category": "One Way"})

In [35]:
for x in oneWayDoc:
    print(x)

{'_id': ObjectId('5bf72c8d8a95a662949378f7'), 'trip_id': 94851140, 'duration': 8, 'start_time': '2018-07-01 00:04:00', 'end_time': '2018-07-01 00:12:00', 'start_station': 3058, 'start_lat': 34.035801, 'start_lon': -118.23317, 'end_station': 3082, 'end_lat': 34.04652, 'end_lon': -118.237411, 'bike_id': 6279, 'plan_duration': 30, 'trip_route_category': 'One Way', 'passholder_type': 'Monthly Pass'}
{'_id': ObjectId('5bf72c8d8a95a662949378f8'), 'trip_id': 94851141, 'duration': 8, 'start_time': '2018-07-01 00:04:00', 'end_time': '2018-07-01 00:12:00', 'start_station': 3058, 'start_lat': 34.035801, 'start_lon': -118.23317, 'end_station': 3082, 'end_lat': 34.04652, 'end_lon': -118.237411, 'bike_id': 6518, 'plan_duration': 365, 'trip_route_category': 'One Way', 'passholder_type': 'Flex Pass'}
{'_id': ObjectId('5bf72c8d8a95a662949378f9'), 'trip_id': 94851138, 'duration': 15, 'start_time': '2018-07-01 00:09:00', 'end_time': '2018-07-01 00:24:00', 'start_station': 4147, 'start_lat': 34.145248, 's

In [51]:
#collection.drop()

In [52]:
mapbox_api_key = "pk.eyJ1IjoiY2hhcmxlaGwiLCJhIjoiY2pvbHM0NzZyMDZoMDNxczg0bXB5ZjV6ZiJ9.MSwFdl5_aHm9jcbDFjnVvA"

In [41]:
from mapbox import Directions

In [70]:
service = Directions(access_token=mapbox_api_key)
# The input waypoints are features, typically GeoJSON-like feature dictionaries
origin = {
    'type': 'Feature',
    'properties': {'name': '7th & Flower'},
    'geometry': {
        'type': 'Point',
        'coordinates': [-118.25854, 34.0485]}}
destination = {
    'type': 'Feature',
    'properties': {'name': 'Ports O\'Call'},
    'geometry': {
        'type': 'Point',
        'coordinates': [-118.27779, 33.73352]}}
# The directions() method can be called with a list of features and the desired profile
response = service.directions([origin, destination], 'mapbox/cycling')

In [80]:
response

<Response [200]>

In [78]:
json.loads(response.text)

{'routes': [{'geometry': 'ebynExjxpU~W{e@z~@t{@fUo[lyBlwAjzA^Dxx@xh@p@Ppc@bj@KlF|\\juAS|OxX~cOk@TdX|{HYjTGDy\\|w@G@|Nhc@vDxiCkCBhNfm@VrVaXhaDkVfMk|@|nBq]',
   'legs': [{'summary': '',
     'weight': 11336.2,
     'duration': 10314.8,
     'steps': [],
     'distance': 41497.8}],
   'weight_name': 'cyclability',
   'weight': 11336.2,
   'duration': 10314.8,
   'distance': 41497.8}],
 'waypoints': [{'name': 'West 7th Street',
   'location': [-118.25853, 34.048511]},
  {'name': 'Nagoya Way', 'location': [-118.277586, 33.733601]}],
 'code': 'Ok',
 'uuid': 'cjotjzcjw012u3to08csh7eig'}